In [1]:
global file
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".
raw_file = file

In [1]:
from datetime import datetime
from datetime import date
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import math
import pandas as pd
import  os as os
import duckdb as db
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
sg = Subgrounds()
holders = sg.load_api('https://api.studio.thegraph.com/query/28103/token-holders/0.0.13')

In [60]:
token_balances = holders.Query.tokenHolderBalances(
    orderBy='timestamp',
    first=1000 ,
    orderdirection = 'asc',
    where=[
            token_balances.date  > '2022-09-24'
          ]
)

In [61]:
balances = sg.query_df([
    token_balances.date,
    token_balances.holder.token,
    token_balances.holder.holder,
    token_balances.balance
])
balances

,tokenHolderBalances_date,tokenHolderBalances_holder_token_id,tokenHolderBalances_holder_token_address,tokenHolderBalances_holder_token_blockchain,tokenHolderBalances_holder_token_name,tokenHolderBalances_holder_holder,tokenHolderBalances_balance
0,2022-09-24T00:03:35.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x2796317b0ff8538f253012862c06787adfb8ceb6,1.235389e+04
1,2022-09-24T00:03:35.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x6571d6be3d8460cf5f7d6711cd9961860029d85f,0.000000e+00
2,2022-09-24T00:03:35.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xf3b844811e5c745143cf4104d7833b427208ddb3,0.000000e+00
3,2022-09-24T00:35:59.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,1.000000e-17
4,2022-09-24T00:35:59.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xc987d503a9f78f6d1d782c1fff5af4cb34437e3f,9.466921e+00
...,...,...,...,...,...,...,...
995,2022-09-26T01:01:11.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x08f68110f1e0ca67c80a24b4bd206675610f445d,4.410036e+01
996,2022-09-26T01:01:11.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xd54e4f8a55247487dd7c8c59f14ee5c0c2889477,4.086038e-03
997,2022-09-26T01:02:35.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xc987d503a9f78f6d1d782c1fff5af4cb34437e3f,7.638779e+00
998,2022-09-26T01:02:35.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xd8c07491caa1edf960db3ceff387426d53942ea0,1.554847e-04


token_balances_exp = holders.Query.tokenHolderBalances(
    orderBy='timestamp',
    orderdirection = 'asc',
    first=1000,
    skip= 1000,
    where=[
            token_balances.date  > '2022-05-05'
          ]
)
balances_exp = sg.query_df([
    token_balances_exp.date,
    token_balances_exp.holder.token,
    token_balances_exp.holder.holder,
    token_balances_exp.balance
])
balances_exp

In [62]:
##ok. We gotta retrieve pretty much every frickin' row. But only 1000 rows at a time, skipping by 1000 with every iteration. Until the number of rows returned is less than 1000. Orash
n=0
done = 0
rowcount = 0
while done == 0:
    n=n+1
    token_balances_2 = holders.Query.tokenHolderBalances(
        orderBy='timestamp',
        first=1000,
        skip = 1000*1+n,
        orderdirection = 'asc',
        where=[
            token_balances.date  > '2022-09-24'
          ]
    )

    balances_2 = sg.query_df([
        token_balances_2.date,
        token_balances_2.holder.token,
        token_balances_2.holder.holder,
        token_balances_2.balance
    ])
    last_date = max(balances['tokenHolderBalances_date'])
    rowcount = len(balances_2)
    print("iteration:",n,"rows:",rowcount, last_date)
    frames = [balances, balances_2]
    balances = pd.concat(frames)
    if rowcount<1000:
        done=1

balances


iteration: 1 rows: 321 2022-09-26T01:11:23.000Z


,tokenHolderBalances_date,tokenHolderBalances_holder_token_id,tokenHolderBalances_holder_token_address,tokenHolderBalances_holder_token_blockchain,tokenHolderBalances_holder_token_name,tokenHolderBalances_holder_holder,tokenHolderBalances_balance
0,2022-09-24T00:03:35.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x2796317b0ff8538f253012862c06787adfb8ceb6,1.235389e+04
1,2022-09-24T00:03:35.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x6571d6be3d8460cf5f7d6711cd9961860029d85f,0.000000e+00
2,2022-09-24T00:03:35.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xf3b844811e5c745143cf4104d7833b427208ddb3,0.000000e+00
3,2022-09-24T00:35:59.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0,1.000000e-17
4,2022-09-24T00:35:59.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xc987d503a9f78f6d1d782c1fff5af4cb34437e3f,9.466921e+00
...,...,...,...,...,...,...,...
316,2022-09-26T18:36:59.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.000000e+00
317,2022-09-26T18:36:59.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0xc987d503a9f78f6d1d782c1fff5af4cb34437e3f,7.175421e+00
318,2022-09-26T18:37:11.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x74894a63ec289c71c641caa45c9688ed4bd93468,7.855046e+00
319,2022-09-26T18:37:11.000Z,gOHM/Ethereum,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,Ethereum,gOHM,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.000000e+00


In [63]:
balances.to_csv("balances.csv", index = False)